## Imports and Vairable Definitions

In [2]:
import os
import tensorflow as tf
# import keras
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.0.0


In [5]:
IMG_DIR = '../furniture-detector/img/'
TRAIN_DIR = IMG_DIR + 'train/'
VAL_DIR =  IMG_DIR + 'val/'
OUTPUT_DIR = '../saved-models/'

IMAGE_SIZE = 224
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)
BATCH_SIZE = 10
EPOCHS = 1

## Set up Image Dataset Generators

In [6]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
  validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    subset='training')

val_generator = datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    subset='validation')

Found 3220 images belonging to 5 classes.
Found 804 images belonging to 5 classes.


## Configure the model

### Import Base Model


In [7]:
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
                        input_shape=IMAGE_SHAPE,
                        include_top=False, 
                        weights='imagenet'
)

base_model.trainable = False

### Create and complile the model

In [8]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(val_generator.num_classes, activation='softmax', name='output_layer_test')
])

# compile model
model.compile(optimizer=tf.keras.optimizers.Adam(), # TODO: Test different optimizers such as RMSProp
              loss='categorical_crossentropy',      # TODO: Test diff loss
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
conv2d (Conv2D)              (None, 5, 5, 32)          368672    
_________________________________________________________________
dropout (Dropout)            (None, 5, 5, 32)          0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 32)                0         
_________________________________________________________________
output_layer_test (Dense)    (None, 5)                 165       
Total params: 2,626,821
Trainable params: 368,837
Non-trainable params: 2,257,984
_________________________________________________________________


## Train the model

In [9]:
history = model.fit_generator(train_generator, 
                              epochs=EPOCHS,
                              validation_data=val_generator
                              )

 10/322 [..............................] - ETA: 15:32 - loss: 2.3624 - accuracy: 0.4300

KeyboardInterrupt: 

## Save the model
Save the model in HDF5 format

In [4]:
shoe_model_file = os.path.join(OUTPUT_DIR, 'base_keras.h5')
tf.keras.models.save_model(model, shoe_model_file)

model.save(shoe_model_file)

train_generator.class_indices

NameError: name 'model' is not defined